# RAG End-to-End – Toy-Notebook (offline)

Vereinfachte RAG-Pipeline ohne externe Modelle: Chunks -> Toy-Embedding -> Retrieval -> Kontext -> Toy-Antwort.

In [1]:
import re
import numpy as np
import pandas as pd
from collections import Counter

np.set_printoptions(suppress=True, precision=3)


## 1) Mini-Korpus und Chunks

In [2]:
docs = [
    {"id": 1, "title": "Frankreich", "text": "Paris ist die Hauptstadt von Frankreich. Frankreich liegt in Europa."},
    {"id": 2, "title": "Deutschland", "text": "Berlin ist die Hauptstadt von Deutschland. Deutschland hat 16 Bundeslaender."},
    {"id": 3, "title": "Italien", "text": "Rom ist die Hauptstadt von Italien. Italien liegt in Suedeuropa."},
    {"id": 4, "title": "Python", "text": "Python ist eine Programmiersprache. Python wird oft fuer Data Science verwendet."},
]

chunks = []
for d in docs:
    sents = [s.strip() for s in d["text"].split(".") if s.strip()]
    for i, sent in enumerate(sents):
        chunks.append({
            "chunk_id": f"{d['id']}-{i}",
            "doc_id": d["id"],
            "title": d["title"],
            "text": sent,
        })

pd.DataFrame(chunks)


,chunk_id,doc_id,title,text
0,1-0,1,Frankreich,Paris ist die Hauptstadt von Frankreich
1,1-1,1,Frankreich,Frankreich liegt in Europa
2,2-0,2,Deutschland,Berlin ist die Hauptstadt von Deutschland
3,2-1,2,Deutschland,Deutschland hat 16 Bundeslaender
4,3-0,3,Italien,Rom ist die Hauptstadt von Italien
5,3-1,3,Italien,Italien liegt in Suedeuropa
6,4-0,4,Python,Python ist eine Programmiersprache
7,4-1,4,Python,Python wird oft fuer Data Science verwendet


## 2) Toy-Embedding (handgemachte Semantik)

Dimensionen: `[stadt, land, hauptstadt, europa, programmierung, daten]`

In [3]:
token_re = re.compile(r"[a-zA-ZäöüÄÖÜß]+")

def tokenize(text):
    return [t.lower() for t in token_re.findall(text)]

SEMVEC = {
    "paris": np.array([1,0,0,0,0,0], float),
    "berlin": np.array([1,0,0,0,0,0], float),
    "rom": np.array([1,0,0,0,0,0], float),
    "frankreich": np.array([0,1,0,1,0,0], float),
    "deutschland": np.array([0,1,0,1,0,0], float),
    "italien": np.array([0,1,0,1,0,0], float),
    "hauptstadt": np.array([0,0,1,0,0,0], float),
    "europa": np.array([0,0,0,1,0,0], float),
    "suedeuropa": np.array([0,0,0,1,0,0], float),
    "liegt": np.array([0,0,0,0.3,0,0], float),
    "python": np.array([0,0,0,0,1,0], float),
    "programmiersprache": np.array([0,0,0,0,1,0], float),
    "data": np.array([0,0,0,0,0.4,0.8], float),
    "science": np.array([0,0,0,0,0.2,0.8], float),
    "verwendet": np.array([0,0,0,0,0.2,0.2], float),
}

def embed(text):
    toks = tokenize(text)
    v = np.zeros(6, dtype=float)
    for t in toks:
        if t in SEMVEC:
            v += SEMVEC[t]
    for t, cnt in Counter(toks).items():
        if t in {"hauptstadt", "frankreich", "deutschland", "italien", "python"} and t in SEMVEC:
            v += 0.2 * cnt * SEMVEC[t]
    n = np.linalg.norm(v)
    return v / n if n > 0 else v

chunk_vectors = np.vstack([embed(c["text"]) for c in chunks])
pd.DataFrame(
    chunk_vectors,
    index=[c["chunk_id"] for c in chunks],
    columns=["stadt","land","hauptstadt","europa","prog","daten"]
).round(3)


,stadt,land,hauptstadt,europa,prog,daten
1-0,0.434,0.520,0.52,0.520,0.000,0.000
1-1,0.000,0.433,0.00,0.902,0.000,0.000
2-0,0.434,0.520,0.52,0.520,0.000,0.000
2-1,0.000,0.707,0.00,0.707,0.000,0.000
3-0,0.434,0.520,0.52,0.520,0.000,0.000
3-1,0.000,0.433,0.00,0.902,0.000,0.000
4-0,0.000,0.000,0.00,0.000,1.000,0.000
4-1,0.000,0.000,0.00,0.000,0.743,0.669


## 3) Retrieval per Cosine Similarity

In [4]:
def retrieve(query, top_k=3):
    q = embed(query)
    sims = chunk_vectors @ q
    rows = []
    for c, s in zip(chunks, sims):
        rows.append({**c, "score": float(s)})
    return pd.DataFrame(rows).sort_values("score", ascending=False).head(top_k), q

query = "Was ist die Hauptstadt von Frankreich?"
retrieved, qvec = retrieve(query, top_k=3)
print("Frage:", query)
retrieved[["chunk_id","title","text","score"]].round(3)


Frage: Was ist die Hauptstadt von Frankreich?


,chunk_id,title,text,score
0,1-0,Frankreich,Paris ist die Hauptstadt von Frankreich,0.901
2,2-0,Deutschland,Berlin ist die Hauptstadt von Deutschland,0.901
4,3-0,Italien,Rom ist die Hauptstadt von Italien,0.901


## 4) Prompt-Kontext bauen

In [5]:
context = "\n".join([f"[{r.chunk_id}] {r.text}" for r in retrieved.itertuples()])
prompt = (
    "Beantworte die Frage nur mit dem gegebenen Kontext.\n\n"
    f"Kontext:\n{context}\n\n"
    f"Frage: {query}\n"
    "Antwort:"
)
print(prompt)


Beantworte die Frage nur mit dem gegebenen Kontext.

Kontext:
[1-0] Paris ist die Hauptstadt von Frankreich
[2-0] Berlin ist die Hauptstadt von Deutschland
[3-0] Rom ist die Hauptstadt von Italien

Frage: Was ist die Hauptstadt von Frankreich?
Antwort:


## 5) Toy-Generator (regelbasiert)

In [6]:
def toy_generate_answer(question, retrieved_df):
    q = question.lower()
    if "text" in retrieved_df.columns:
        text_blob = " ".join(retrieved_df["text"].astype(str).tolist()).lower()
    else:
        text_blob = ""

    if "hauptstadt" in q and "frankreich" in q and "paris" in text_blob:
        return "Paris ist die Hauptstadt von Frankreich."
    if "hauptstadt" in q and "deutschland" in q and "berlin" in text_blob:
        return "Berlin ist die Hauptstadt von Deutschland."
    if "hauptstadt" in q and "italien" in q and "rom" in text_blob:
        return "Rom ist die Hauptstadt von Italien."
    if "python" in q and ("data science" in text_blob or ("python" in text_blob and "verwendet" in text_blob)):
        return "Python wird oft fuer Data Science verwendet."
    return "Ich kann die Antwort aus dem gefundenen Kontext nicht sicher ableiten."

print("Antwort:", toy_generate_answer(query, retrieved))


Antwort: Paris ist die Hauptstadt von Frankreich.


## 6) Ohne Retrieval (nur Frage)

In [7]:
print("Antwort ohne Kontext:", toy_generate_answer(query, pd.DataFrame(columns=["text"])))


Antwort ohne Kontext: Ich kann die Antwort aus dem gefundenen Kontext nicht sicher ableiten.


## 7) Mehrere Beispielanfragen

In [8]:
for q in [
    "Was ist die Hauptstadt von Deutschland?",
    "Was ist die Hauptstadt von Italien?",
    "Wofuer wird Python oft verwendet?",
]:
    top, _ = retrieve(q, top_k=2)
    print("\nFrage:", q)
    print(top[["chunk_id","title","score"]].round(3).to_string(index=False))
    print("Toy-Antwort:", toy_generate_answer(q, top))



Frage: Was ist die Hauptstadt von Deutschland?
chunk_id       title  score
     1-0  Frankreich  0.901
     2-0 Deutschland  0.901
Toy-Antwort: Berlin ist die Hauptstadt von Deutschland.

Frage: Was ist die Hauptstadt von Italien?
chunk_id       title  score
     1-0  Frankreich  0.901
     2-0 Deutschland  0.901
Toy-Antwort: Ich kann die Antwort aus dem gefundenen Kontext nicht sicher ableiten.

Frage: Wofuer wird Python oft verwendet?
chunk_id  title  score
     4-0 Python   0.99
     4-1 Python   0.83
Toy-Antwort: Python wird oft fuer Data Science verwendet.


## 8) Einordnung

- Struktur entspricht RAG: Frage -> Embedding -> Retrieval -> Kontext -> Generierung.
- Das Embedding ist hier didaktisch vereinfacht und offline reproduzierbar.